In [2]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB

import sys
sys.path.append('../')
import saedfsc

Here is a notional list of suppliers.

In [3]:
saedfsc.suppliers

,Name,Location,Rating
0,Supplier1,Location4,1.961931
1,Supplier2,Location1,4.843323
2,Supplier3,Location2,2.775760
3,Supplier4,Location3,2.757291
4,Supplier5,Location2,1.551448
5,Supplier6,Location3,1.926430
6,Supplier7,Location4,3.271909


In [13]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

Create model container.

In [7]:
m = gp.Model()

{'rearwing': {909: <gurobi.Var x[rearwing][909]>,
  396: <gurobi.Var x[rearwing][396]>,
  885: <gurobi.Var x[rearwing][885]>,
  1255: <gurobi.Var x[rearwing][1255]>,
  1354: <gurobi.Var x[rearwing][1354]>,
  2591: <gurobi.Var x[rearwing][2591]>,
  1642: <gurobi.Var x[rearwing][1642]>,
  28: <gurobi.Var x[rearwing][28]>,
  2532: <gurobi.Var x[rearwing][2532]>,
  1858: <gurobi.Var x[rearwing][1858]>,
  680: <gurobi.Var x[rearwing][680]>,
  748: <gurobi.Var x[rearwing][748]>,
  335: <gurobi.Var x[rearwing][335]>,
  2106: <gurobi.Var x[rearwing][2106]>,
  1381: <gurobi.Var x[rearwing][1381]>,
  299: <gurobi.Var x[rearwing][299]>,
  1669: <gurobi.Var x[rearwing][1669]>,
  540: <gurobi.Var x[rearwing][540]>,
  1226: <gurobi.Var x[rearwing][1226]>,
  1143: <gurobi.Var x[rearwing][1143]>},
 'frontwing': {909: <gurobi.Var x[frontwing][909]>,
  396: <gurobi.Var x[frontwing][396]>,
  885: <gurobi.Var x[frontwing][885]>,
  1255: <gurobi.Var x[frontwing][1255]>,
  1354: <gurobi.Var x[frontwing][135

Create variables

In [16]:
x = {} # part order quantity variables
for subsystem in partOptions.keys():
    x[subsystem] = m.addVars(partOptions[subsystem].index.values, name="x[" + subsystem + "]")
y = m.addVars(saedfsc.suppliers['Name'].to_list(), vtype=GRB.BINARY, name="y") # supplier selection variables